In [14]:
#importing user movie rating data from Movie lens
import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': 'a52717a052f343a1b9020e4160a61913',
    'region': 'dallas',
    'user_id': 'c8d82fa9a27147ba8a9c1d984f5185d3',
    'username': 'member_157b57e8fa7b7b37664471ffe4919c0ba44b9c4f',
    'password': 'L7{xz9VVCq^6k80s'
}

configuration_name = 'os_4f7c12d0b952450db479df6fdbb30d48_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
user_ratings_raw_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .load(bmos.url('FindHotelProjecttemp', 'mock_recommendation_movies_ratings.csv'))
user_ratings_raw_df.show(5)
print "_c0 = MovieId, _c1 = UserId, _c2 = Rating"

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  1|  3|4.0|
|  1|  6|5.0|
|  1|  8|4.0|
|  1| 10|4.0|
|  1| 11|4.5|
+---+---+---+
only showing top 5 rows

_c0 = MovieId, _c1 = UserId, _c2 = Rating


In [15]:
#sampling dataset for training, validation and testing the ALS model
ratings_data = user_ratings_raw_df.rdd.map(lambda x: (x[1], x[0], x[2]))
trainingSet_RDD, validationSet_RDD = ratings_data.randomSplit([7, 3], seed=0L)
validation_predict_RDD = validationSet_RDD.map(lambda x: (x[0], x[1]))

In [16]:
#building the ALS model
#identifying best rank for the matrix
from pyspark.mllib.recommendation import ALS
import math
seed = 1L
iterations = 10
regularization_parameter = 0.1
ranks = [8,10,12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(trainingSet_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
    predictions = model.predictAll(validation_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_preds = validationSet_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank
print ('The best model was trained with rank %s' % best_rank)

For rank 8 the RMSE is 0.80927081895
For rank 10 the RMSE is 0.813837078685
For rank 12 the RMSE is 0.81453263768
The best model was trained with rank 8


In [17]:
#saving the trained model 

model.save(sc,'AlsModel')
print 'Model successfully saved'

Model successfully saved


# Collaborative Filtering 

In [18]:
#importing required libraries

import pyspark
import itertools
from pyspark.sql import HiveContext
import unicodedata
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.functions import sort_array
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [19]:
#this function is used to arrive at combinations of user-movies
def comb(user,xx):
    combinations=[[(x,a) for x in user] for a in xx]
    return combinations

#this function is used for unlisting
def unllist(xx):
    unl=list(itertools.chain.from_iterable(xx))
    return unl

In [20]:
#setting a threshold value of the number of users
num=1 

#Loading ALS Model
ALS_model = MatrixFactorizationModel.load(sc, "AlsModel")

In [22]:
#subset of users and their rated movieids and ratings
user_orig=user_ratings_raw_df.rdd.map(lambda x:(x[1],x[0],x[2]))#rdd of user ratings
user_ori_df=user_orig.toDF(["userid","movieid","rating"])#Dataframe of user ratings
userpart = user_ori_df.where(user_ori_df.userid<=num)
print userpart.show(5)

+------+-------+------+
|userid|movieid|rating|
+------+-------+------+
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
+------+-------+------+
only showing top 5 rows

None


In [23]:
#filtered users and the rated movie IDs only
userpartrdd=userpart.rdd.map(lambda x: (x[0], x[1]))

#filtering selected users their rated movieids and the corresponding ratings
userpartrdd1 = userpart.rdd.map(lambda x: (x[0], x[1], x[2]))
print userpartrdd1.take(5)

[(u'1', u'47', u'3.5'), (u'1', u'50', u'3.5'), (u'1', u'112', u'3.5'), (u'1', u'151', u'4.0'), (u'1', u'223', u'4.0')]


In [24]:
#finding the unique set of movieIDs

oriMovSet=user_ori_df.agg(F.sort_array(F.collect_set("movieid")))
print oriMovSet.show(5)
oriMovSetrdd=oriMovSet.rdd
oriMovrdd=oriMovSetrdd.map(lambda x:x[0])

+--------------------------------------+
|sort_array(collect_set(movieid), true)|
+--------------------------------------+
|                  [1, 10, 100, 1000...|
+--------------------------------------+

None


In [25]:
#generating all possible combinations

userrange=range(1,num+1)
MovUser_comb=oriMovrdd.map(lambda x: comb(userrange,x))

#unlisting
unrdd=MovUser_comb.flatMap(lambda x:unllist(x))

#finding user unrated movies
Newusers=unrdd.subtract(userpartrdd)

In [26]:
#ALS to predict for unrated movies
test_user_all_recommendations = ALS_model.predictAll(Newusers)

#Combining the userid, movieid, predicted rating and existing rating
all_userdata = test_user_all_recommendations.union(userpartrdd1)

In [104]:
#Converting to Dataframe
all_userdataDF=all_userdata.toDF(["user","product","rating"])

#Sorting DataFrame
all_userdataDF2=all_userdataDF.sort(asc("user"),asc("product")).na.drop().limit(4160)
print all_userdataDF2.show(5)

+----+-------+------------------+
|user|product|            rating|
+----+-------+------------------+
|   1|      1|3.7856571923064712|
|   1|     10| 3.509212126086192|
|   1|     11|3.4147215875345025|
|   1|     14| 3.006741214481731|
|   1|     15| 3.036109581296813|
+----+-------+------------------+
only showing top 5 rows

None


In [105]:
#collecting per product the userids and their corresponding rating
all_userdataDF1=all_userdataDF2.groupby("user").agg(F.collect_set("product"),F.collect_list("rating"))

#Converting to rdd
all_userdatadf1rdd=all_userdataDF1.rdd.map(lambda x: (x[0],x[1],x[2]))

#Converting to dense Vector
all_userDV=all_userdatadf1rdd.map(lambda x:(x[0],Vectors.dense(x[2])))
all_userDV.take(1)

[(1,
  DenseVector([3.7857, 3.5092, 3.4147, 3.0067, 3.0361, 3.4699, 3.4578, 3.3886, 2.6972, 2.7663, 3.2492, 3.2256, 3.1089, 3.1312, 3.2846, 3.3809, 3.2664, 3.5938, 2.9793, 3.0344, 3.7992, 2.982, 4.0011, 3.0991, 3.0012, 3.054, 3.3317, 3.2647, 3.3749, 2.902, 3.1263, 3.3646, 2.933, 2.66, 3.0696, 2.9394, 3.2901, 3.3564, 2.7863, 3.8475, 3.5026, 3.3393, 2.7213, 3.3019, 2.6651, 3.0076, 3.02, 3.2406, 3.6363, 3.3898, 3.091, 2.989, 2.7124, 3.4609, 3.5355, 3.5206, 3.0127, 3.4329, 3.1027, 3.5929, 2.9827, 3.5264, 2.6793, 2.3895, 2.9432, 2.7268, 3.0551, 3.6189, 3.6474, 3.4259, 2.9815, 2.8296, 3.0758, 3.3597, 3.3287, 3.4542, 3.7345, 2.768, 2.1808, 2.9142, 4.1943, 3.2667, 3.3384, 3.3158, 3.0494, 3.76, 3.1781, 2.7784, 3.0594, 4.1194, 3.1695, 3.14, 3.4415, 2.4567, 3.4249, 3.0896, 3.0211, 3.1976, 3.4376, 3.257, 2.8922, 3.7319, 2.5396, 3.738, 3.348, 3.7304, 3.0403, 3.2259, 2.9863, 2.4798, 3.0532, 3.4224, 2.9365, 3.5738, 3.093, 3.1854, 2.6884, 2.2384, 2.6524, 3.1367, 2.7898, 2.9727, 3.3685, 3.2819, 3.7372,

This is the user rating predictions for new movies

# Feature Extraction (Content Based)

In [43]:
#importing the IMDB movie data

movie_content = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .load(bmos.url('FindHotelProjecttemp', 'mock_recommendation_movies_content.csv'))
movie_content.show(5)
print "_c0 = MovieId, _c1 = directorName, _c2 = actor2name, _c3 = actor3name, _c4 =  actor1name "

+---+---------------+-----------------+--------------------+--------------+
|_c0|            _c1|              _c2|                 _c3|           _c4|
+---+---------------+-----------------+--------------------+--------------+
|  1|  John Lasseter|John Ratzenberger|           Tom Hanks|    Jim Varney|
| 10|Martin Campbell|    Joe Don Baker|   Izabella Scorupco|  Tchéky Karyo|
| 11|     Rob Reiner|  Samantha Mathis|      Shawna Waldron| Wendie Malick|
| 14|   Oliver Stone|      Bob Hoskins|     Anthony Hopkins|    Joan Allen|
| 15|   Renny Harlin|   Frank Langella|Christopher Maste...|Matthew Modine|
+---+---------------+-----------------+--------------------+--------------+
only showing top 5 rows

_c0 = MovieId, _c1 = directorName, _c2 = actor2name, _c3 = actor3name, _c4 =  actor1name 


In [44]:
#importing the required libraries
import pyspark
from pyspark.sql import HiveContext
import unicodedata
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [45]:
#this function is used to combine the extracted features to the users
def tricomb(user,xx,xy):
    combinations=[(user[i],xx[i],xy[i]) for i in range(len(user))] 
    return combinations 
    
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def comb(user,xx):
    combinations=[(x,xx) for x in user] 
    return combinations 

In [75]:
#removing null values and converting to rdd
movie_content_df = movie_content.na.drop()
movie_content_rdd = movie_content_df.rdd.map(lambda x: (x[0], x[1], x[2], x[3], x[4]))

In [76]:
#preparing the director and artist names to appear as a set of sentences for feature extraction

Data_m=movie_content_rdd.map(lambda x: (x[0], [''.join(x[1].split(' ')),''.join(x[3].split(' ')),''.join(x[4].split(' '))]))
f_data=Data_m.map(lambda x:(x[0],' '.join(x[1])))
unicode_remo1=f_data.map(lambda x: (x[0],unicodedata.normalize('NFKD', x[1]).encode('ascii','ignore')))
unicode_remolo=unicode_remo1.map(lambda x:(x[0], x[1].lower()))
sparkDF = unicode_remolo.map(lambda x: (x[1],)).toDF(['sentences'])
sparkDF.show(5)

+--------------------+
|           sentences|
+--------------------+
|johnlasseter tomh...|
|martincampbell iz...|
|robreiner shawnaw...|
|oliverstone antho...|
|rennyharlin chris...|
+--------------------+
only showing top 5 rows



In [77]:
#Hashing vectorizer and TF-IDF

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import IndexToString
tokenizer = Tokenizer(inputCol="sentences", outputCol="words")
wordsData = tokenizer.transform(sparkDF)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [80]:
#extracting the vectorised features by the index value and combining them with the movie

reScdatafl=rescaledData.rdd.map(lambda x: (f7(x[1]),x[3]))
pp=reScdatafl.map(lambda x:(x[0],len(x[0]),list(x[1].indices),len(list(x[1].indices)),list(x[1].values),len(list(x[1].values))))
Relevant=pp.map(lambda x:(x[0],x[2],x[4]))
Relevant_unlist=Relevant.map(lambda x: tricomb(x[0],x[1],x[2]))
Rel_unli_fl=Relevant_unlist.flatMap(lambda x: x)
unicode_remolwrds=unicode_remo1.map(lambda x:(x[0],f7(x[1].split(' '))))
unicode_unlis=unicode_remolwrds.map(lambda x: comb(x[1],x[0]))
unicode_unlisfl=unicode_unlis.flatMap(lambda x: x)
featIndMov=unicode_unlisfl.zip(Rel_unli_fl).map(lambda (x,y):(x[1],y[0],y[1],y[2]))
featIndMovDF=featIndMov.map(lambda x: (int(x[0]),x[1],int(x[2]),float(x[3]))).toDF(["movieid","feature","index","value"])
movieunique = featIndMovDF.select("movieid").distinct()
featIndMovDF.show(5)

+-------+----------------+------+-----------------+
|movieid|         feature| index|            value|
+-------+----------------+------+-----------------+
|      1|    johnlasseter| 84464|7.640363528422996|
|      1|        tomhanks| 95553|5.155456878634996|
|      1|       jimvarney| 98665|6.541751239754886|
|     10|  martincampbell| 93581|6.136286131646722|
|     10|izabellascorupco|101203|7.234898420314831|
+-------+----------------+------+-----------------+
only showing top 5 rows



In [81]:
#Transposing and creating dense feature vector
from pyspark.ml.feature import VectorAssembler

featIndMovDF_transposed = featIndMovDF.groupby('feature').pivot('movieid').max('value').fillna(0)
columns = featIndMovDF_transposed.columns
columns1 = columns[1:]
assembler = VectorAssembler(inputCols=columns1, outputCol="vector")
vector_output = assembler.transform(featIndMovDF_transposed)
vector_output1 = vector_output.select(["vector", "feature"])
featureSparceVector_rdd=vector_output1.rdd.map(lambda x: (x[0],x[1]))
print featureSparceVector_rdd.take(5)

[(SparseVector(4160, {3489: 7.6404}), u'finolahughes'), (SparseVector(4160, {2479: 6.3876, 2825: 7.6404, 3020: 7.2349, 3759: 7.6404, 4041: 7.6404}), u'dominiccooper'), (SparseVector(4160, {2148: 7.6404}), u'pauldaggett'), (SparseVector(4160, {937: 7.6404}), u'bobmarley'), (SparseVector(4160, {1079: 6.9472, 3484: 6.5418, 3577: 6.9472}), u'tarsemsingh')]


# Matrix Dot Product

In [106]:
#converting to MLlib vector

from pyspark.ml import linalg as ml_linalg
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

In [108]:
#Matrix Dot product
from pyspark.mllib.linalg.distributed import *

final_rdd = featureSparceVector_rdd.cartesian(all_userDV)
final_model = final_rdd.map(lambda x: (as_mllib(x[0][0]).dot(Vectors.dense(x[1][1]))))
print final_model.take(100)

[27.641140814482295, 120.22636284542156, 22.557823437316948, 23.429583500125204, 66.660700298758499, 23.31194253343531, 50.541362504719928, 46.457110405456042, 47.486542210356987, 69.020471065900125, 64.020799974592848, 26.527545290917676, 26.326946318415445, 268.50563892793934, 32.937680223377839, 24.458257644553854, 128.84145450231932, 109.89760364559928, 113.79217243129197, 24.189992592388698, 44.287031432416953, 26.152595182236119, 48.466984039628244, 35.733863557934953, 45.317233401499848, 25.689144232914941, 41.785520215266871, 43.823421468682632, 21.77526346840142, 24.764827631057859, 22.200992450157937, 55.134230164721998, 23.117435080568978, 25.354726982119555, 82.887708404157451, 21.719565454174361, 27.010432315512215, 24.74104724186784, 21.700573723728215, 28.837173654229638, 21.595118140823434, 26.724616951689974, 25.860190181916291, 21.363893541851695, 16.857542724023251, 98.772823609880447, 16.828980313295052, 48.166749588717479, 347.07204199687828, 24.946832862160498, 69

This is the user feature affinity value